<a href="https://colab.research.google.com/github/anadiedrichs/2023-nlp-tps/blob/main/2_bot_tfidf_nltk_genero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK

Usaré como corpus el sitio web de la unidad de Género y Diversidad de la Universidad Tecnológica Nacional, Facultad Regional Mendoza

https://www4.frm.utn.edu.ar/genero-y-diversidad/




In [1]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
#nltk.download("punkt")
#nltk.download("wordnet")
#nltk.download('omw-1.4')

# Download the Spanish stopwords
nltk.download("stopwords")

# Download the Spanish tokenizers
nltk.download("punkt")

# Download the Spanish lemmatizer data
nltk.download("omw")
# Download the Spanish corpora
nltk.download("cess_esp")  # Corpus of Contemporary Spanish
nltk.download("conll2002")  # Spanish Named Entity Recognition (NER) Corpus


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

### Datos
Se consumirán los datos del artículo de wikipedia sobre el deporte "Tennis" en inglés.

In [2]:
raw_html = urllib.request.urlopen('https://www4.frm.utn.edu.ar/genero-y-diversidad/')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [3]:
import requests
from bs4 import BeautifulSoup

# URL de la página web
url = 'https://www4.frm.utn.edu.ar/genero-y-diversidad/'

# Realizar una solicitud HTTP para obtener el contenido de la página
response = requests.get(url)

# Comprobar si la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML de la página con Beautiful Soup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer todo el texto del HTML
    all_text = soup.get_text()

    # Imprimir el texto
    print(all_text)
else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")






 

genero y diversidad - UTN - Facultad Regional Mendoza


















































































 





















 Search




 Close










































 















 
















 
















 
















 
















 
















 





















 








 


























Comisión Asesoradel Programa de Género Regional 











¿CUÁNDO SURGE LA COMISIÓN ASESORA DE GÉNERO Y DIVERSIDAD? 




La Universidad Tecnológica Nacional Facultad Regional Mendoza crea la Comisión Asesora del Programa de
Género Local mediante la resolución 699/2019 CD UTN FRM, en concordancia con la Ley Nº 27.229 “Ley Micaela
de Capacitación Obligatoria en Género para Todas las Personas que Integran los Tres Poderes del Estado” y la
Ordenanza del Consejo Superior UTN nº 1638 titulada «Protocolo de Acción Institucional para la Prevención e
Intervención ante situaciones de Violencia de Género u 

In [4]:
# Demos un vistazo
article_text = all_text.lower()

In [5]:
print("Cantidad de caracteres en el sitio web principal:", len(article_text))

Cantidad de caracteres en el sitio web principal: 6955


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [6]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres:
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!   .'

In [7]:
# Demos un vistazo
text

' genero y diversidad - utn - facultad regional mendoza search close comisión asesoradel programa de género regional ¿cuándo surge la comisión asesora de género y diversidad? la universidad tecnológica nacional facultad regional mendoza crea la comisión asesora del programa de género local mediante la resolución 699/2019 cd utn frm, en concordancia con la ley nº 27.229 “ley micaela de capacitación obligatoria en género para todas las personas que integran los tres poderes del estado” y la ordenanza del consejo superior utn nº 1638 titulada «protocolo de acción institucional para la prevención e intervención ante situaciones de violencia de género u orientación sexual». ¿cuáles son las funcionesde la comisión asesora de género y diversidad? las principales funciones de la comisión son: – recibir consultas o denuncias de aquellas situaciones contempladas por el protocolo y realizar las entrevistas personales a las personas afectadas – llevar registro de los casos y de las intervenciones 

In [8]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 5776


### 3 - Dividir el texto en sentencias y en palabras

In [9]:
#from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Tokenize Spanish text
#text = "Este es un ejemplo de texto en español."
#tokens = word_tokenize(text, language='spanish')

# Remove stopwords
spanish_stopwords = list(stopwords.words('spanish'))
#filtered_tokens = [word for word in tokens if word.lower() not in spanish_stopwords]

# Perform other NLP tasks as needed (lemmatization, POS tagging, etc.)


In [10]:
corpus = nltk.sent_tokenize(text, language='spanish') # divide en oraciones
words = nltk.word_tokenize(text, language='spanish') # divide en términos

In [11]:
# Demos un vistazo
corpus[:130]

[' genero y diversidad - utn - facultad regional mendoza search close comisión asesoradel programa de género regional ¿cuándo surge la comisión asesora de género y diversidad?',
 'la universidad tecnológica nacional facultad regional mendoza crea la comisión asesora del programa de género local mediante la resolución 699/2019 cd utn frm, en concordancia con la ley nº 27.229 “ley micaela de capacitación obligatoria en género para todas las personas que integran los tres poderes del estado” y la ordenanza del consejo superior utn nº 1638 titulada «protocolo de acción institucional para la prevención e intervención ante situaciones de violencia de género u orientación sexual».',
 '¿cuáles son las funcionesde la comisión asesora de género y diversidad?',
 'las principales funciones de la comisión son: – recibir consultas o denuncias de aquellas situaciones contempladas por el protocolo y realizar las entrevistas personales a las personas afectadas – llevar registro de los casos y de las in

In [12]:
# Demos un vistazo
words[:50]

['genero',
 'y',
 'diversidad',
 '-',
 'utn',
 '-',
 'facultad',
 'regional',
 'mendoza',
 'search',
 'close',
 'comisión',
 'asesoradel',
 'programa',
 'de',
 'género',
 'regional',
 '¿cuándo',
 'surge',
 'la',
 'comisión',
 'asesora',
 'de',
 'género',
 'y',
 'diversidad',
 '?',
 'la',
 'universidad',
 'tecnológica',
 'nacional',
 'facultad',
 'regional',
 'mendoza',
 'crea',
 'la',
 'comisión',
 'asesora',
 'del',
 'programa',
 'de',
 'género',
 'local',
 'mediante',
 'la',
 'resolución',
 '699/2019',
 'cd',
 'utn',
 'frm']

In [13]:
print("Vocabulario:", len(words))

Vocabulario: 903


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal), language='spanish'))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=spanish_stopwords)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar

    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- denuncias
- micaela
- formulario
- instagram


In [16]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

In [17]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True,share=True)

<ipython-input-17-2ee8e5560b73>:9: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c8b08c8294f8c0a803.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Q: denuncias


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


A: 1) a través del correo electrónico:generoutnmendoza@gmail.com 2) a través del siguiente formulario formulario las denuncias pueden ser efectuadas por cualquier persona a la que asistenlos derechos vulneradossegún las situaciones comprendidas, ya seadirectamente o por haberconocido o presenciadoalguna de las situaciones descriptas.durante la recepción delas denuncias, segarantiza: - respeto yconfidencialidad.- gratuidad en todo elproceso administrativo.- contención.- la no revictimización.- diligencia y celeridaden el proceso.
Q: protocolo
A: ¿qué es un protocolo?
Q: facebook
A: para acceder directamente a ella puedes usar este enlace: https://www4.frm.utn.edu.ar/genero-y-diversidad/ también se puede acceder mediante el menú principal del sitio, ... previous next información universidad tecnológica nacional facultad regional mendoza rodriguez 273, ciudad mendoza cp (m5502aje) república argentina trámites por mesa de entrada: e-mail: mesadeentrada@frm.utn.edu.ar​ canales digitales fac

### Alumno

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.